### Construindo um Agente 'React' simples do zero
Building a Simple React Agent from Scratch

In [1]:
# https://til.simonwillison.net/llms/python-react-pattern

In [1]:
# carregar o arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [2]:
import openai
import re
import httpx # alternativa a biblioteca requests
import os

In [3]:
from openai import OpenAI
client = OpenAI()

In [7]:
# Teste

model = "gpt-4o-mini"
prompt = "Hello, chatgpt!"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[{'role':'user', 'content': prompt}]
)

In [8]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [9]:
chat_completion

ChatCompletion(id='chatcmpl-AIFHqDBDVtXJ5ayrvWnhnTQRfWNEX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728912030, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_8552ec53e1', usage=CompletionUsage(completion_tokens=9, prompt_tokens=13, total_tokens=22, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0), prompt_tokens_details={'cached_tokens': 0}))

### Criando a classe do Agente

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []

        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, prompt):
        self.messages.append({"role": "user", "content": prompt})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self, model="gpt-4o-mini", temperature=0):
        completion = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=self.messages
        )
        return completion.choices[0].message.content

# Criando o prompt 'ReAct'

In [ ]:
# Exemplo de prompt: https://smith.langchain.com/hub/hwchase17/react

In [7]:
prompt = '''
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5

You then output:

Answer: A pen costs $5


Example session #2

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
'''.strip()

### Criando as 'Tools' (Ferramentas)

In [13]:
# 1. the calculate() function takes in a string, evaluates that string, and returns the result
def calculate(what):
    return eval(what)

# 2. the get_cost() function returns the cost for a pen, a book, and a stapler
def get_cost(thing):
    if thing in 'pen': 
        return('A pen costs $5')
    elif thing in 'book':
        return('A book costs $20')
    elif thing in 'stapler':
        return('A stapler costs $10')
    else:
        return('A random thing for writing costs $12.')

# 3. the wikipedia() function uses the Wikipedia API to search for a specific query on Wikipedia
def wikipedia(q):
    response = httpx.get('https://en.wikipedia.org/w/api.php', params={
        'action': 'query',
        'list': 'search',
        'srsearch': q,
        'format': 'json'
    })
    results = response.json().get('query').get('search', [])
    
    if not results:
        return None
    return results[0]['snippet']

In [17]:
# dictionary that maps the function names to the functions themselves
known_actions = {
    'calculate': calculate,
    'get_cost': get_cost,
    'wikipedia': wikipedia
}

### Testando o Agente

In [18]:
# Exemplo 1
my_agent = Agent(prompt)

In [19]:
# calling the agent with this initial question
result = my_agent('How much does a pen cost?')
print(result)

Thought: I should look up the cost of a pen using the get_cost action.  
Action: get_cost: pen  
PAUSE


In [20]:
# creating the next prompt that will be used as an observation and passed to the language model
next_prompt = f"Observation: {get_cost('pen')}"

In [21]:
# calling the agent with that next prompt
my_agent(next_prompt)

'Answer: A pen costs $5.'

In [25]:
my_agent.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then output:\n\nAnswer: 

In [26]:
# Exemplo 2
abot = Agent(prompt)

In [27]:
question = '''I want to buy a pen and a book. 
How much do they cost in total?'''

abot(question)

"Thought: I need to find the cost of both a pen and a book to calculate the total cost. I'll start by getting the cost of a pen and then the cost of a book. \nAction: get_cost: pen\nPAUSE"

In [28]:
# executing the action and creating the next prompt. 
next_prompt = f'Observation: {get_cost("pen")}'
print(next_prompt)

Observation: A pen costs $5


In [29]:
abot(next_prompt)

'Thought: Now that I know the cost of the pen, I need to find out the cost of the book to calculate the total. \nAction: get_cost: book\nPAUSE'

In [30]:
next_prompt = f'Observation: {get_cost("book")}'
print(next_prompt)

Observation: A book costs $20


In [31]:
abot(next_prompt)

'Thought: I have the costs for both the pen and the book. The pen costs $5 and the book costs $20. Now, I can calculate the total cost. \nAction: calculate: 5 + 20\nPAUSE'

In [32]:
abot(next_prompt)

'Answer: The total cost of a pen and a book is $25.'

In [33]:
# Exemplo 3
abot = Agent(prompt)

In [34]:
query = '2024 United Kingdom elections'
abot(query)

'Thought: I should look up information about the 2024 United Kingdom elections on Wikipedia to get the latest details.  \nAction: wikipedia: 2024 United Kingdom elections  \nPAUSE'

In [35]:
# calling the wikipedia() function and creating the next prompt
next_prompt = f'Observation: {wikipedia(query)}'
print(next_prompt)

Observation: The <span class="searchmatch">2024</span> <span class="searchmatch">United</span> <span class="searchmatch">Kingdom</span> general <span class="searchmatch">election</span> was held on Thursday, 4 July <span class="searchmatch">2024</span> to elect 650 members of Parliament to the House of Commons, the lower house


In [36]:
# calling the agent with the new prompt, which in turn is giving the final answer
abot(next_prompt)

'Answer: The 2024 United Kingdom general election was held on Thursday, 4 July 2024, to elect 650 members of Parliament to the House of Commons.'

In [37]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then output:\n\nAnswer: 

### Automatizando o Agente

In [38]:
# defining a regex for finding the action string
action_re = re.compile(r'^Action: (\w+): (.*)$')  # python regular expression to select Action:

In [39]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

         # using the regex to parse the response from the agent.
        actions = [ # This is a list comprehension
            action_re.match(a) for a in result.split('\n') if action_re.match(a)
        ]

        if actions:
            action, action_input = actions[0].groups() 

            if action not in known_actions:
                raise Exception(f'Unknown action: {action}: {action_input}')

            print(f' -- running {action} {action_input}')
            observation = known_actions[action](action_input) 
           
            print(f'Observation: {observation}')
            next_prompt = f'Observation: {observation}'
        else:
            return

In [40]:
question = '''I want to buy 2 books and 3 pens. How much do I have to pay?'''
query(question)

Thought: I need to find the cost of a book and a pen to calculate the total cost for 2 books and 3 pens. I will first look up the cost of a book and then the cost of a pen. 

Action: get_cost: book
PAUSE
 -- running get_cost book
Observation: A book costs $20
Thought: Now that I know the cost of a book is $20, I need to find the cost of a pen to complete the calculation for the total cost of 2 books and 3 pens.

Action: get_cost: pen
PAUSE
 -- running get_cost pen
Observation: A pen costs $5
Thought: I now have the costs: a book costs $20 and a pen costs $5. I can calculate the total cost for 2 books and 3 pens using these prices.

Action: calculate: 2 * 20 + 3 * 5
PAUSE
 -- running calculate 2 * 20 + 3 * 5
Observation: 55
Answer: You have to pay $55 for 2 books and 3 pens.


In [41]:
question = '''UEFA EURO 2024'''
query(question)

Thought: I should look up UEFA EURO 2024 on Wikipedia to gather information about the event.  
Action: wikipedia: UEFA EURO 2024  
PAUSE
 -- running wikipedia UEFA EURO 2024  
Observation: <span class="searchmatch">2024</span> <span class="searchmatch">UEFA</span> European Football Championship, commonly referred to as <span class="searchmatch">UEFA</span> <span class="searchmatch">Euro</span> <span class="searchmatch">2024</span> (stylised as <span class="searchmatch">UEFA</span> <span class="searchmatch">EURO</span> <span class="searchmatch">2024</span>) or simply <span class="searchmatch">Euro</span> <span class="searchmatch">2024</span>, was the 17th <span class="searchmatch">UEFA</span> European
Answer: UEFA EURO 2024 is the 17th UEFA European Football Championship.
